<a href="https://colab.research.google.com/github/Aleena24/srgan_cloud/blob/main/srgan_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()  # Upload your Kaggle API key (kaggle.json)

# Move the Kaggle API key to the appropriate directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Install the Kaggle package if not already installed
!pip install kaggle

# Download the dataset
!kaggle datasets download -d apollo2506/eurosat-dataset

# Unzip the dataset
!unzip eurosat-dataset.zip -d ./EuroSat


Streaming output truncated to the last 5000 lines.
  inflating: ./EuroSat/EuroSATallBands/River/River_1990.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_1991.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_1992.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_1993.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_1994.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_1995.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_1996.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_1997.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_1998.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_1999.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_2.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_20.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_200.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_2000.tif  
  inflating: ./EuroSat/EuroSATallBands/River/River_2001.tif  
  inflating: ./EuroSat/Eu

In [10]:
!pip install highlight_text
!pip install perlin_noise
!pip install noise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for noise: filename=noise-1.2.2-cp310-cp310-linux_x86_64.whl size=56278 sha256=8e287d01bb72bdb4bbec9668eaf6ea7bc710fb9aa8c1d220855d2fae145c73d2
  Stored in directory: /root/.cache/pip/wheels/b2/01/64/ca2f6ed556c204d5ce2d3719a58757a5504aeb5525654692d0
Successfully built noise


In [ ]:
import numpy as np
import pandas as pd
import random
from os import listdir, mkdir, rename
from os.path import join, exists
import shutil
import datetime
import matplotlib.pyplot as plt
from highlight_text import ax_text, fig_text
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from perlin_noise import PerlinNoise
from noise import pnoise2

# Image preparation
classes = listdir('./EuroSat/EuroSAT')
path_target = './EuroSat/all_targets'
path_input = './EuroSat/all_inputs'

# Create directories if they do not exist
if not exists(path_input):
    mkdir(path_input)
if not exists(path_target):
    mkdir(path_target)

k = 1
for kind in classes:
    path = join('./EuroSat/EuroSAT', str(kind))
    try:
        files = listdir(path)
    except NotADirectoryError:
        continue  # Skip this iteration if path is not a directory

    for i, f in enumerate(files):
        if f.endswith('.jpg') or f.endswith('.png'):  # Filter only image files
            shutil.copyfile(join(path, f), join(path_target, f))
            rename(join(path_target, f), join(path_target, f'{k}.jpg'))
            k += 1

def generate_perlin_noise(width, height, scale, octaves, persistence, lacunarity):
    noise = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            noise[i][j] = pnoise2(i / scale,
                                  j / scale,
                                  octaves=octaves,
                                  persistence=persistence,
                                  lacunarity=lacunarity,
                                  repeatx=width,
                                  repeaty=height,
                                  base=0)
    return noise

def normalize_noise(noise):
    min_val = noise.min()
    max_val = noise.max()
    return (noise - min_val) / (max_val - min_val)

def generate_clouds(width, height, base_scale, octaves, persistence, lacunarity):
    clouds = np.zeros((height, width))
    for octave in range(1, octaves + 1):
        scale = base_scale / octave
        layer = generate_perlin_noise(width, height, scale, 1, persistence, lacunarity)
        clouds += layer * (persistence ** octave)

    clouds = normalize_noise(clouds)
    return clouds

def overlay_clouds(image, clouds, alpha=0.5):
    clouds_rgb = np.stack([clouds] * 3, axis=-1)
    image = image.astype(float) / 255.0
    clouds_rgb = clouds_rgb.astype(float)
    blended = image * (1 - alpha) + clouds_rgb * alpha
    blended = (blended * 255).astype(np.uint8)
    return blended

width, height = 64, 64
octaves = 12  # number of noise layers combined
persistence = 0.5  # lower persistence reduces the amplitude of higher-frequency octaves
lacunarity = 2  # higher lacunarity increases the frequency of higher-frequency octaves

for i in range(len(listdir(path_target))):
    base_scale = random.uniform(5, 120)  # noise frequency
    alpha = random.uniform(0, 1)  # transparency

    clouds = generate_clouds(width, height, base_scale, octaves, persistence, lacunarity)
    img = np.asarray(Image.open(join(path_target, f'{i+1}.jpg')))
    image = Image.fromarray(overlay_clouds(img, clouds, alpha))
    image.save(join(path_input, f'{i+1}.jpg'))
    print(f'Processed {i+1}/{len(listdir(path_target))}')

# Display a random image pair
idx = np.random.randint(len(listdir(path_target)))
fig, ax = plt.subplots(1, 2)
ax[0].imshow(np.asarray(Image.open(join(path_target, f'{idx}.jpg'))))
ax[1].imshow(np.asarray(Image.open(join(path_input, f'{idx}.jpg'))))
ax[0].set_title("Target")
ax[0].axis('off')
ax[1].set_title("Input")
ax[1].axis('off')
plt.show()


Streaming output truncated to the last 5000 lines.
Processed 3842/27000
Processed 3843/27000
Processed 3844/27000
Processed 3845/27000
Processed 3846/27000
Processed 3847/27000
Processed 3848/27000
Processed 3849/27000
Processed 3850/27000
Processed 3851/27000
Processed 3852/27000
Processed 3853/27000
Processed 3854/27000
Processed 3855/27000
Processed 3856/27000
Processed 3857/27000
Processed 3858/27000
Processed 3859/27000
Processed 3860/27000
Processed 3861/27000
Processed 3862/27000
Processed 3863/27000
Processed 3864/27000
Processed 3865/27000
Processed 3866/27000
Processed 3867/27000
Processed 3868/27000
Processed 3869/27000
Processed 3870/27000
Processed 3871/27000
Processed 3872/27000
Processed 3873/27000
Processed 3874/27000
Processed 3875/27000
Processed 3876/27000
Processed 3877/27000
Processed 3878/27000
Processed 3879/27000
Processed 3880/27000
Processed 3881/27000
Processed 3882/27000
Processed 3883/27000
Processed 3884/27000
Processed 3885/27000
Processed 3886/27000
Proc